In [37]:
from __future__ import annotations

from pathlib import Path
WORK_DIR = Path.cwd()

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import smtplib

## Testing Alpha Vantage API

In [ ]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='UX2OT39HH6HK14LB')
data, meta_data = ts.get_intraday('AAPL')

In [42]:
import numpy as np
import pandas as pd
import requests
import time

class AlphaVantageDataLoader():
    def __init__(
                 self, 
                 key: str):
        self.key=key

    
    def load_stock_data(self,
        tickers: list = [],
        frequency: str = 'daily', 
        output_size: str = 'full'
        ) -> dict: 

        freq_dict = {'intraday': 'TIME_SERIES_INTRADAY',
                     'daily': 'TIME_SERIES_DAILY',
                     'weekly': 'TIME_SERIES_WEEKLY',
                     'monthly': 'TIME_SERIES_MONTHLY'}
        
        if type(tickers) == str:
            tickers = [tickers]

        start = time.time()
        meta_dict = {} 
        df_dict = {}
        count=0
        
        for num, ticker in tqdm(enumerate(tickers)):
            link = f'https://www.alphavantage.co/query?function={freq_dict[frequency]}&symbol={ticker}&outputsize={output_size}&apikey={self.key}'
            stock = requests.get(link)
            keys_list = list(stock.json().keys())
            
            try:
                data, metadata = stock.json()[keys_list[-1]], stock.json()[keys_list[0]]

                df = pd.DataFrame(data).T
                df.columns = [col.split(' ')[-1] for col in df.columns]
                df = df.apply(pd.to_numeric)

                meta_dict[ticker] = meta_data
                df_dict[ticker]= df
                count+=1
            except:
                pass

            #Alpha Vantage only allows 5 API call per minute
            if (count+1)%5==0: 
                end = time.time()
                wait_time = (start+60) - end
                time.sleep(wait_time)
                start = time.time()
        
        self.meta = meta_dict

        return df_dict


In [38]:
ticker = pd.read_csv(WORK_DIR/'TSX.txt', sep='\t')

In [43]:
import random

ticker_list = ticker.loc[(~ticker.Symbol.str.contains('PR'))&
                           (~ticker.Description.str.contains('ETF'))&
                           (~ticker.Description.str.contains('Fund'))&
                           (~ticker.Description.str.contains('hdg'))]['Symbol'].tolist()

sampled_ticker = random.sample(ticker_list, 10)

In [44]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stock_data = loader.load_stock_data(tickers=sampled_ticker)

10it [02:00, 12.00s/it]


In [47]:
sampled_ticker

['XTG.TO',
 'ET.TO',
 'NVO.WT.TO',
 'SGQ.TO',
 'BAM.PF.H.TO',
 'AHC.WT.TO',
 'LWRK.TO',
 'LEV.TO',
 'PSD.TO',
 'BBD.B.TO']

In [46]:
stock_data.keys()

dict_keys(['XTG.TO', 'ET.TO', 'SGQ.TO', 'PSD.TO'])

In [65]:
link = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol=PSD.TO&apikey=UX2OT39HH6HK14LB'
stock = requests.get(link)

In [66]:
stock.json()

{}

In [43]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stock_data = loader.load_stock_data(tickers=['DOL.TO', 'BB.TO'])

In [47]:
list(stock_data.values())[0]

,open,high,low,close,volume
2021-05-27,53.6600,54.0150,52.5900,52.7000,4031224
2021-05-26,53.5100,54.3200,53.1300,53.7000,828000
2021-05-25,53.6100,53.7100,52.7100,53.3100,725000
2021-05-21,54.2600,54.2800,53.3200,53.3800,588800
2021-05-20,53.4300,54.3700,53.2500,54.2300,826600
...,...,...,...,...,...
2009-10-16,3.1267,3.1267,3.0433,3.0833,2872200
2009-10-15,3.1583,3.1650,3.1133,3.1200,2167200
2009-10-14,3.1817,3.2050,3.1183,3.1583,2547000
2009-10-13,3.2500,3.2733,3.1667,3.1767,6267000


#### Sending Notification Email

In [30]:
sender = "tuateststock@outlook.com" 
receiver = "wongsangaroon@gmail.com"
password = ("ImTestingStocks123") 
message = f'From: {sender}\nTo: {receiver}\nSubject: Stock Alert!!\n\nI Made It WORKKK!!\n'

In [ ]:

server = smtplib.SMTP('smtp-mail.outlook.com', 587)
server.starttls()
server.login(sender, password)
print("Login Success")
server.sendmail(sender, receiver, msg = message) 
print("Email was sent")
server.close()